In [1]:
import yaml
import psycopg2
from yaml.loader import SafeLoader
from sqlalchemy import create_engine   
from database_utils import DatabaseConnector
import pandas as pd
from sqlalchemy import inspect
from database_utils import DatabaseConnector
from data_extraction import DataExtractor
from data_cleaning import DataCleaning

In [174]:
def read_db_creds(creds_filepath):
    with open(creds_filepath) as f:
        creds = yaml.load(f, Loader=SafeLoader)
        return creds
    
read_db_creds('db_creds.yaml')

{'RDS_HOST': 'data-handling-project-readonly.cq2e8zno855e.eu-west-1.rds.amazonaws.com',
 'RDS_PASSWORD': 'AiCore2022',
 'RDS_USER': 'aicore_admin',
 'RDS_DATABASE': 'postgres',
 'RDS_PORT': 5432}

In [175]:
creds = read_db_creds('db_creds.yaml')
def init_db_engine(creds):
    engine = create_engine(f"postgresql://{creds['RDS_USER']}:{creds['RDS_PASSWORD']}@{creds['RDS_HOST']}:{creds['RDS_PORT']}/{creds['RDS_DATABASE']}")
    return engine.connect()

In [176]:
run_engine = init_db_engine(creds)
def list_db_tables():
        inspector = inspect(run_engine)
        table_names = inspector.get_table_names()
        return table_names

list_db_tables()

Exception during reset or similar
Traceback (most recent call last):
  File "/home/van28/anaconda3/envs/my_env/lib/python3.10/site-packages/sqlalchemy/pool/base.py", line 763, in _finalize_fairy
    fairy._reset(pool, transaction_was_reset)
  File "/home/van28/anaconda3/envs/my_env/lib/python3.10/site-packages/sqlalchemy/pool/base.py", line 1038, in _reset
    pool._dialect.do_rollback(self)
  File "/home/van28/anaconda3/envs/my_env/lib/python3.10/site-packages/sqlalchemy/engine/default.py", line 683, in do_rollback
    dbapi_connection.rollback()
psycopg2.OperationalError: SSL SYSCALL error: EOF detected



['legacy_store_details', 'legacy_users', 'orders_table']

In [177]:
def read_rds_tables(table_names):
    df = pd.read_sql_table(table_names[1],run_engine)
    return df

In [178]:
table_names = list_db_tables()
table = read_rds_tables(table_names)
print(table.head(5))

   index first_name last_name date_of_birth                    company  \
0      0   Sigfried     Noack    1990-09-30         Heydrich Junitz KG   
1      1        Guy     Allen    1940-12-01                    Fox Ltd   
2      2      Harry  Lawrence    1995-08-02  Johnson, Jones and Harris   
3      3     Darren   Hussain    1972-09-23                Wheeler LLC   
4      4      Garry     Stone    1952-12-20                 Warner Inc   

                  email_address  \
0             rudi79@winkler.de   
1  rhodesclifford@henderson.com   
2  glen98@bryant-marshall.co.uk   
3    daniellebryan@thompson.org   
4       billy14@long-warren.com   

                                             address         country  \
0                       Zimmerstr. 1/0\n59015 Gießen         Germany   
1  Studio 22a\nLynne terrace\nMcCarthymouth\nTF0 9GH  United Kingdom   
2                 92 Ann drive\nJoanborough\nSK0 6LR  United Kingdom   
3             19 Robinson meadow\nNew Tracy\nW22 2QG  Un

In [39]:
string_table = table.to_string('users.txt')

In [7]:
table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15320 entries, 0 to 15319
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   index          15320 non-null  int64 
 1   first_name     15320 non-null  object
 2   last_name      15320 non-null  object
 3   date_of_birth  15320 non-null  object
 4   company        15320 non-null  object
 5   email_address  15320 non-null  object
 6   address        15320 non-null  object
 7   country        15320 non-null  object
 8   country_code   15320 non-null  object
 9   phone_number   15320 non-null  object
 10  join_date      15320 non-null  object
 11  user_uuid      15320 non-null  object
dtypes: int64(1), object(11)
memory usage: 1.4+ MB


In [44]:
table.dtypes

index             int64
first_name       object
last_name        object
date_of_birth    object
company          object
email_address    object
address          object
country          object
country_code     object
phone_number     object
join_date        object
user_uuid        object
dtype: object

In [28]:
print(table.columns)
print(len(table.columns))

Index(['index', 'first_name', 'last_name', 'date_of_birth', 'company',
       'email_address', 'address', 'country', 'country_code', 'phone_number',
       'join_date', 'user_uuid'],
      dtype='object')
12


In [179]:
# How many NULL values?
count_null = 0
for index, row in table.iterrows():
    if row['first_name'] == 'NULL':
        count_null +=1
print(count_null)

21


In [44]:
null_values = table[table['first_name'].isnull()]
print(null_values)

Empty DataFrame
Columns: [index, first_name, last_name, date_of_birth, company, email_address, address, country, country_code, phone_number, join_date, user_uuid]
Index: []


In [180]:
# Solution: Eliminate NULL values

table_null = table[table.first_name != 'NULL']
print(len(table_null))
print(len(table))

15299
15320


In [181]:
# Deal with date formats
users_dates = table[['date_of_birth','join_date']]

In [77]:
users_dates.to_string('user_dates.txt')

In [182]:
# Solution: 
# Results in cleaned_dates.txt
import datetime
format = '%Y-%m-%d'

cleaned_dates = users_dates.apply(pd.to_datetime, infer_datetime_format=True, errors='coerce')
cleaned_dates.to_string('cleaned_dates.txt')

In [185]:
date_cols = table[['date_of_birth','join_date']]
users_df_clean = date_cols.apply(pd.to_datetime, 
                                          infer_datetime_format=True, # Fix dates
                                          errors='coerce')
users_df_clean.to_string('cleaned_dates.txt')

In [94]:
# Remove /n 
address_col = table['address']
address_col.to_string('address.txt')

In [132]:
address_col = table['address']
print(address_col.iloc[0])
print(address_col.iloc[0].replace('\n',' '))

Zimmerstr. 1/0
59015 Gießen
Zimmerstr. 1/0 59015 Gießen


In [151]:
# Solution: remove spaces \n
address_col = address_col.apply(lambda x:x.replace('\n', ','))
address_col.to_string('address_clean.txt')

In [166]:
address_col.to_csv('address_clean.csv')

In [172]:
check_address = pd.read_csv('address_clean.csv')
print(check_address['address'].iloc[113])

Studio 89W,Laura junctions,West Nicolashire,B0 5FU


In [2]:
# Card Data
from data_extraction import DataExtractor

In [40]:
link = 'https://data-handling-public.s3.eu-west-1.amazonaws.com/card_details.pdf'
data_extractor = DataExtractor()
credit_card_df = data_extractor.retrieve_pdf_data(link)
print(credit_card_df.iloc[0])

card_number                            30060773296197
expiry_date                                     09/26
card_provider             Diners Club / Carte Blanche
date_payment_confirmed                     2015-11-25
Name: 0, dtype: object


In [33]:
print(len(credit_card_df))

15309


In [41]:
credit_card_df = credit_card_df[credit_card_df.card_number != 'NULL']
print(len(credit_card_df))

15298


In [6]:
credit_card_df.iloc[0]['card_provider']

'Diners Club / Carte Blanche'

In [7]:
credit_card_df.iloc[0]['card_provider'].replace(' / ','/')

'Diners Club/Carte Blanche'

In [42]:
# Fix Indexing
index = [row for row in range(0,len(credit_card_df))]
credit_card_df['new_index'] = index

In [43]:
credit_card_df.set_index(['new_index'])

,card_number,expiry_date,card_provider,date_payment_confirmed
new_index,,,,
0,30060773296197,09/26,Diners Club / Carte Blanche,2015-11-25
1,349624180933183,10/23,American Express,2001-06-18
2,3529023891650490,06/23,JCB 16 digit,2000-12-26
3,213142929492281,09/27,JCB 15 digit,2011-02-12
4,502067329974,10/25,Maestro,1997-03-13
...,...,...,...,...
15293,180036921556789,12/28,JCB 15 digit,1997-06-06
15294,180018030448512,11/24,JCB 15 digit,2004-06-16
15295,3569953313547220,04/24,JCB 16 digit,2020-02-05


In [37]:
credit_card_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15298 entries, 0 to 18
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   card_number             15298 non-null  object
 1   expiry_date             15298 non-null  object
 2   card_provider           15298 non-null  object
 3   date_payment_confirmed  15298 non-null  object
 4   new_index               15298 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 717.1+ KB


In [12]:
print(credit_card_df.iloc[157]['card_number'])
print(credit_card_df.iloc[158]['card_number'])
print(credit_card_df.iloc[0]['card_number'])
print(credit_card_df.iloc[157]['card_number'].isnumeric())
print(credit_card_df.iloc[158]['card_number'].isnumeric())
print(credit_card_df.iloc[0]['card_number'].isnumeric())

???3554954842403828
3550482967595170
30060773296197
False
True


AttributeError: 'int' object has no attribute 'isnumeric'

In [44]:
credit_card_df['card_number'] = credit_card_df['card_number'].astype('str')

In [29]:
print(credit_card_df.iloc[157]['card_number'])
print(credit_card_df.iloc[158]['card_number'])
print(credit_card_df.iloc[0]['card_number'])
print(credit_card_df.iloc[157]['card_number'].isnumeric())
print(credit_card_df.iloc[158]['card_number'].isnumeric())
print(credit_card_df.iloc[0]['card_number'].isnumeric())

???3554954842403828
3550482967595170
30060773296197
False
True
True


In [45]:
print(len(credit_card_df))
credit_card_df = credit_card_df[credit_card_df['card_number'].str.isnumeric()]
print(len(credit_card_df))

15298
15258


In [46]:
credit_card_df['card_provider'].unique()

array(['Diners Club / Carte Blanche', 'American Express', 'JCB 16 digit',
       'JCB 15 digit', 'Maestro', 'Mastercard', 'Discover',
       'VISA 19 digit', 'VISA 16 digit', 'VISA 13 digit'], dtype=object)

In [9]:
import requests
import json
def list_number_of_stores(number_of_stores_endpoint=
                          'https://aqj7u5id95.execute-api.eu-west-1.amazonaws.com/prod/number_stores',
                              header_dict={'x-api-key':'yFBQbwXe9J3sd6zWVAMrK6lcxxr0q1lr2PT6DDMX'}):
        number_of_stores = requests.get(number_of_stores_endpoint, headers=header_dict).content
        number_of_stores = json.loads(number_of_stores)
        return number_of_stores['number_stores']

In [10]:
store_list = list_number_of_stores()
store_list

451

In [7]:
# only retrieves one store
def retrieve_a_store_data(store_endpoint=
                             'https://aqj7u5id95.execute-api.eu-west-1.amazonaws.com/prod/store_details/',
                             header_dict={'x-api-key':'yFBQbwXe9J3sd6zWVAMrK6lcxxr0q1lr2PT6DDMX'}):
        list_of_stores = []
        store_number = list_number_of_stores()['number_stores']-1
        store = requests.get(f'{store_endpoint}{store_number}', headers=header_dict).content
        list_of_stores.append(json.loads(store))
        return list_of_stores

In [10]:
from async_get import http_get
import asyncio
import aiohttp
import time
from random import randint

In [13]:
async def get_random_pokemon_name():
    pokemon_id = randint(1,898)
    pokemon_url = f"https://pokeapi.co/api/v2/pokemon/{pokemon_id}"
    pokemon = await http_get(pokemon_url)
    return str(pokemon['name'])

async def main():
    pokemon_name = await get_random_pokemon_name()
    print(pokemon_name)

In [14]:
asyncio.run(main())

zamazenta


In [2]:
import asyncio
import aiohttp
import time

websites = """https://www.youtube.com
https://www.facebook.com
https://www.baidu.com
https://www.yahoo.com
https://www.amazon.com
https://www.wikipedia.org
http://www.qq.com
https://www.google.co.in
https://www.twitter.com
https://www.live.com
http://www.taobao.com
https://www.bing.com
https://www.instagram.com
http://www.weibo.com
http://www.sina.com.cn
https://www.linkedin.com
http://www.yahoo.co.jp
http://www.msn.com
http://www.uol.com.br
https://www.google.de
http://www.yandex.ru
http://www.hao123.com
https://www.google.co.uk
https://www.reddit.com
https://www.ebay.com
https://www.google.fr
https://www.t.co
"""


async def get(url, session):
    try:
        async with session.get(url=url) as response:
            resp = await response.read()
            print("Successfully got url {} with resp of length {}.".format(url, len(resp)))
    except Exception as e:
        print("Unable to get url {} due to {}.".format(url, e.__class__))


async def main(urls):
    async with aiohttp.ClientSession() as session:
        ret = await asyncio.gather(*[get(url, session) for url in urls])
    print("Finalized all. Return is a list of len {} outputs.".format(len(ret)))


urls = websites.split("\n")
start = time.time()
asyncio.run(main(urls))
end = time.time()

print("Took {} seconds to pull {} websites.".format(end - start, len(urls)))

Unable to get url  due to <class 'aiohttp.client_exceptions.InvalidURL'>.
Successfully got url https://www.wikipedia.org with resp of length 75715.
Successfully got url https://www.google.co.in with resp of length 14129.
Successfully got url https://www.google.co.uk with resp of length 14196.
Successfully got url https://www.google.de with resp of length 14762.
Successfully got url http://www.qq.com with resp of length 132677.
Successfully got url https://www.google.fr with resp of length 14710.
Successfully got url http://www.uol.com.br with resp of length 773016.
Successfully got url https://www.bing.com with resp of length 107389.
Successfully got url https://www.live.com with resp of length 38088.
Successfully got url https://www.t.co with resp of length 0.
Successfully got url https://www.youtube.com with resp of length 740350.
Successfully got url https://www.facebook.com with resp of length 62075.
Successfully got url http://www.msn.com with resp of length 47149.
Successfully go

In [3]:
#stop async loops
import nest_asyncio
nest_asyncio.apply()

In [13]:
def retrieve_stores_data(store_endpoint=
                             'https://aqj7u5id95.execute-api.eu-west-1.amazonaws.com/prod/store_details/',
                             header_dict={'x-api-key':'yFBQbwXe9J3sd6zWVAMrK6lcxxr0q1lr2PT6DDMX'}):
        list_of_stores = []
        store_number = list_number_of_stores()-1
        for number in range(0,store_number):
            store = requests.get(f'{store_endpoint}{number}', headers=header_dict).content
            list_of_stores.append(json.loads(store))
            stores_df = pd.DataFrame(list_of_stores)
        return stores_df

In [14]:

stores_df = retrieve_stores_data()


In [15]:
# fix index
stores_df.columns

Index(['index', 'address', 'longitude', 'lat', 'locality', 'store_code',
       'staff_numbers', 'opening_date', 'store_type', 'latitude',
       'country_code', 'continent'],
      dtype='object')

In [16]:
def clean_store_data(stores_df):
        # Get rid of duplicates
        stores_df = stores_df.drop_duplicates()
        # Drop NULL values
        stores_df = stores_df[stores_df.address != 'N/A']
        # Get rid of rows with wrong values
        stores_df = stores_df.loc[stores_df['country_code'].isin(['GB','US','DE'])]
        # Remove escape characters
        stores_df.loc[:,'address'] =  stores_df.loc[:,'address'].apply(lambda x:x.replace('\n', ','))
        # Fix date formatting
        stores_df.loc[:,'opening_date'] = \
        stores_df.loc[:,'opening_date'].apply(pd.to_datetime,
                                            infer_datetime_format=True,
                                            errors='coerce')
        # FIx misspelling
        stores_df[['continent']] = stores_df[['continent']].apply(lambda x:x.replace('eeEurope','Europe'))
        return  stores_df


In [17]:
clean_store_df = clean_store_data(stores_df)
clean_store_df.to_string('clean_store_data.txt')

/tmp/ipykernel_343216/3753010604.py:11: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  stores_df.loc[:,'opening_date'] = \


In [20]:
clean_store_df[clean_store_df['staff_numbers'].astype('str').apply(lambda x : x.isdigit() == False)]

,index,address,longitude,lat,locality,store_code,staff_numbers,opening_date,store_type,latitude,country_code,continent
31,31,"Flat 69,Suzanne walk,East Michelle,E80 8HS, Ea...",52.68333,None,East Dereham,EA-24B31935,J78,2012-11-09,Outlet,0.93333,GB,Europe
179,179,"Girschnerweg 163,93597 Angermünde, Dahlem",52.4581,None,Dahlem,DA-ACC520AE,30e,1994-03-07,Local,13.28702,DE,Europe
248,248,"7430 Howe Extensions Suite 299,Kellyside, WA 9...",39.71734,None,Sicklerville,SI-C489938D,80R,1994-02-28,Outlet,-74.96933,US,America
341,341,"Studio 8,Lydia groves,North Hilarymouth,IV4 6E...",50.79205,None,Southsea,SO-B5B9CB3B,A97,2018-05-08,Super Store,-1.08593,GB,Europe
375,375,"Salzstraße 1/9,74209 Bad Freienwalde, Charlott...",52.53048,None,Charlottenburg-Nord,CH-99475026,3n9,1995-03-05,Local,13.29371,DE,Europe


In [28]:
import re
clean_store_df.loc[:,'staff_numbers'] = clean_store_df.loc[:,'staff_numbers'].astype('str').apply(lambda x : re.sub('\D','',x))
clean_store_df.to_string('*.txt')

In [38]:
stores_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 450 entries, 0 to 449
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   index          450 non-null    int64 
 1   address        450 non-null    object
 2   longitude      450 non-null    object
 3   lat            11 non-null     object
 4   locality       450 non-null    object
 5   store_code     450 non-null    object
 6   staff_numbers  450 non-null    object
 7   opening_date   450 non-null    object
 8   store_type     450 non-null    object
 9   latitude       449 non-null    object
 10  country_code   450 non-null    object
 11  continent      450 non-null    object
dtypes: int64(1), object(11)
memory usage: 42.3+ KB


In [51]:
stores_df[['continent']].apply(lambda x:x.replace('eeEurope','Europe')).to_string('continent.txt')

In [74]:
stores_df.iloc[:,1:]

,address,longitude,lat,locality,store_code,staff_numbers,opening_date,store_type,latitude,country_code,continent
0,N/A,N/A,N/A,N/A,WEB-1388012W,325,2010-06-12,Web Portal,None,GB,Europe
1,"Flat 72W\nSally isle\nEast Deantown\nE7B 8EB, ...",51.62907,None,High Wycombe,HI-9B97EE4E,34,1996-10-25,Local,-0.74934,GB,Europe
2,"Heckerstraße 4/5\n50491 Säckingen, Landshut",48.52961,None,Landshut,LA-0772C7B9,92,2013-04-12,Super Store,12.16179,DE,Europe
3,"5 Harrison tunnel\nSouth Lydia\nWC9 2BE, Westbury",51.26,None,Westbury,WE-1DE82CEE,69,2014-01-02,Super Store,-2.1875,GB,Europe
4,Studio 6\nStephen landing\nSouth Simon\nB77 2W...,53.0233,None,Belper,BE-18074576,35,2019-09-09,Local,-1.48119,GB,Europe
...,...,...,...,...,...,...,...,...,...,...,...
445,"Flat 7\nStephanie lake\nMorrisside\nHP8 8LH, C...",50.76306,None,Cowes,CO-473A9FBB,94,2008-06-08,Super Store,-1.29772,GB,Europe
446,"Täschestraße 25\n39039 Nördlingen, Kirchlengern",52.2,None,Kirchlengern,KI-78096E8C,61,2005-05-12,Super Store,8.63333,DE,Europe
447,K0ODETRLS3,K8CXLZDP07,UXMWDMX1LC,3VHFDNP8ET,9D4LK7X4LZ,D23PCWSM6S,36IIMAQD58,NN04B3F6UQ,JZP8MIJTPZ,B3EH2ZGQAV,1WZB1TE1HL
448,"Studio 8\nMoss mall\nWest Linda\nM0E 6XR, High...",51.62907,None,High Wycombe,HI-EEA7AE62,33,1998-05-14,Local,-0.74934,GB,Europe


In [79]:
address ='s3://data-handling-public/products.csv'
BUCKET_NAME, OBJECT_NAME = address.split('/')[-2], address.split('/')[-1]
print(BUCKET_NAME, OBJECT_NAME)
        

data-handling-public products.csv


In [18]:
# extract products data
import boto3
def extract_from_s3(address ='s3://data-handling-public/products.csv'):
        BUCKET_NAME, OBJECT_NAME = address.split('/')[-2], address.split('/')[-1]
        print(BUCKET_NAME, OBJECT_NAME)
        s3 = boto3.client(service_name ='s3')
        s3.download_file(BUCKET_NAME,OBJECT_NAME,'products.csv')
        products_df = pd.read_csv('products.csv')
        return products_df
df = extract_from_s3()

data-handling-public products.csv


In [111]:
import re
units_to_remove = ['kg','g','ml']
result=df.loc[:0,'weight'].apply(lambda x:re.sub('[kgml]','',x))
print(result)

0    1.6
Name: weight, dtype: object


In [120]:

df.loc[0:,'weight']=df.loc[0:,'weight'].astype('str').apply(lambda x:re.sub('[kgml]','',x))
print(df)


      Unnamed: 0                                 product_name product_price  \
0              0  FurReal Dazzlin' Dimples My Playful Dolphin        £39.99   
1              1          Tiffany's World Day Out At The Park        £12.99   
2              2          Tiffany's World Pups Picnic Playset         £7.00   
3              3     Tiffany's World Wildlife Park Adventures        £12.99   
4              4                      Cosatto Cosy Dolls Pram        £30.00   
...          ...                                          ...           ...   
1848        1848    Goodmans Qi Autosense In Car Phone Holder        £15.00   
1849        1849    Goodmans Qi Wireless Charging Phone Stand        £15.00   
1850        1850                     RAC 12V Wet & Dry Vacuum        £18.00   
1851        1851    Goodmans Qi Autosense In Car Phone Holder        £15.00   
1852        1852               Beldray Telescopic Ladder 2.9m        £69.00   

     weight        category            EAN  date_ad

In [75]:
x_columns = df['weight'][df['weight'].str.contains('x')]
x_columns

298     12 x 100
299      8 x 150
300     12 x 100
301     12 x 100
302     12 x 100
303      6 x 412
304      6 x 400
305     12 x 100
381       8 x 85
382     40 x 100
383      12 x 85
384       8 x 85
385     40 x 100
386      12 x 85
1562       3 x 2
1563       3 x 2
1571      3 x 90
1668     16 x 10
1669     16 x 10
1676     16 x 10
1677     16 x 10
1689     3 x 132
1690     5 x 145
1692     4 x 400
1700     3 x 132
1701     5 x 145
1727     2 x 200
1752     2 x 200
Name: weight, dtype: object

In [79]:
x_columns.apply(lambda x : x.replace(' x ',' '))

298     12 100
299      8 150
300     12 100
301     12 100
302     12 100
303      6 412
304      6 400
305     12 100
381       8 85
382     40 100
383      12 85
384       8 85
385     40 100
386      12 85
1562       3 2
1563       3 2
1571      3 90
1668     16 10
1669     16 10
1676     16 10
1677     16 10
1689     3 132
1690     5 145
1692     4 400
1700     3 132
1701     5 145
1727     2 200
1752     2 200
Name: weight, dtype: object

In [ ]:
x_columns.apply(lambda x : x.replace(' x ',' '))

In [134]:
# multiple values
def multiply_weight_values(value):
    if 'x' in value:
        value = value.replace(' x ',' ')
        num1, num2 = value.split(' ')[0], value.split(' ')[1]
        new_value = int(num1)*int(num2)
        return new_value
    else:
        return value

In [130]:
sample = '12 x 5'
print(multiply_weight_values(sample))

60


In [133]:
df.loc[0:,'weight'].apply(lambda x: multiply_weight_values(x)).to_string('*.txt')

In [190]:
def convert_product_weights(products_df):
        # Drop NULL values
        products_df = products_df[products_df.weight != 'nan']
        products_df = products_df.dropna()
        # remove kg, g, ml
        products_df.loc[0:,'weight'] = products_df.loc[0:,'weight'].astype('str').apply(lambda x:re.sub('[kgml]','',x))
        # multiply values if contains 'num x num'
        def multiply_weight_values(value):
            if 'x' in value:
                value = value.replace(' x ',' ')
                num1, num2 = value.split(' ')[0], value.split(' ')[1]
                new_value = int(num1)*int(num2)
                return new_value
            else:
                return value
        products_df.loc[0:,'weight'] = products_df.loc[0:,'weight'].apply(lambda x: multiply_weight_values(x)) 
        # drop non numerical values
        '''products_df[products_df['weight'].astype('str').apply(lambda x:x.replace('.','',1).isnumeric())]'''
        
        return products_df

In [189]:
convert_product_weights(df).to_string('$.txt')

In [181]:
df = convert_product_weights(df)
df.loc[0:4]

,Unnamed: 0,product_name,product_price,weight,category,EAN,date_added,uuid,removed,product_code
0,0,FurReal Dazzlin' Dimples My Playful Dolphin,£39.99,1.6,toys-and-games,7425710935115,2005-12-02,83dc0a69-f96f-4c34-bcb7-928acae19a94,Still_avaliable,R7-3126933h
1,1,Tiffany's World Day Out At The Park,£12.99,0.48,toys-and-games,487128731892,2006-01-09,712254d7-aea7-4310-aff8-8bcdd0aec7ff,Still_avaliable,C2-7287916l
2,2,Tiffany's World Pups Picnic Playset,£7.00,590,toys-and-games,1945816904649,1997-03-29,b089ef6f-b628-4e37-811d-fffe0102ba64,Still_avaliable,S7-1175877v
3,3,Tiffany's World Wildlife Park Adventures,£12.99,540,toys-and-games,1569790890899,2013-03-20,d55de422-8b98-47d6-9991-e4bc4c5c0cb0,Removed,D8-8421505n
4,4,Cosatto Cosy Dolls Pram,£30.00,1.91,toys-and-games,7142740213920,2007-12-23,7945b657-cb02-4cc5-96cf-f65ed0a8f235,Still_avaliable,B6-2596063a


In [182]:
df['weight'].loc[10:15]

10    1.478
11      1.2
12      1.2
13     0.66
14     0.91
15      1.8
Name: weight, dtype: object

In [197]:
# drop non numerical values
df[df['weight'].astype('str').apply(lambda x:x.replace('.','').isdigit())].to_string('*.txt')

In [212]:
x = '77g .'
if x[-1].isalnum() is False:
    wrong_char = x[-1]
    print(x.replace(wrong_char,'').strip())


77g


In [223]:
x = '16oz'
if 'oz' in x:
    new_x = x.replace('oz','')
    new_x = float(new_x) * 28.3495
    print(round(new_x,2))


453.59


In [256]:
x = '7g'
print(x[-1]) # grabs units
print(x[-2].isdigit()) # if True, then g (False, then kg)
print(x[:-2].isdigit()) # check for '5 x 70g'


g
True
False


In [269]:
x[-2:]

'ml'

In [271]:
x = '70ml'
if x[-1] == 'g' and x[-2].isdigit() and x[:-2].isdigit() or x[-2:] == 'ml':
    x = x.replace('g','').replace('ml','')
    x = float(x) /1000
    print(x)
else:
    print('not g')

0.07


In [19]:
import re
def convert_product_weights(products_df):
        # ensure correct format e.g. '0.77g .'
        def correct_format(value):
            if value[-1].isalnum() is False:
                wrong_char = value[-1]
                value= value.replace(wrong_char,'').strip()
            return value
        products_df.loc[0:,'weight'] =products_df.loc[0:,'weight'].astype('str').apply(lambda x:correct_format(x))
        # convert oz
        def convert_oz(value):
            if 'oz' in value:
                value = value.replace('oz','')
                value = float(value) * 28.3495
            return value
        products_df.loc[0:,'weight'] = products_df.loc[0:,'weight'].astype('str').apply(lambda x:convert_oz(x))
        # convert grams to kg
        def convert_grams(value):
            if value[-1] == 'g' and value[-2].isdigit() and value[:-2].isdigit()or value[-2:] == 'ml':
                value = value.replace('g','').replace('ml','')
                value = int(value) /1000
            return value
        products_df.loc[0:,'weight'] = products_df.loc[0:,'weight'].astype('str').apply(lambda x:convert_grams(x))
        # remove kg sign
        products_df.loc[0:,'weight'] = products_df.loc[0:,'weight'].astype('str').apply(lambda x:re.sub('[kg]','',x))
        # multiply values if contains 'num x num'
        def multiply_weight_values(value):
            if 'x' in value:
                value = value.replace(' x ',' ')
                num1, num2 = value.split(' ')[0], value.split(' ')[1]
                new_value = int(num1)*int(num2) / 1000
                return new_value
            else:
                return value
        products_df.loc[0:,'weight'] = products_df.loc[0:,'weight'].apply(lambda x: multiply_weight_values(x)) 
        # drop non numerical values
        products_df = products_df[products_df['weight'].astype('str').apply(lambda x:x.replace('.','').isdigit())] # works
        products_df.loc[0:,'weight'] = products_df.loc[0:,'weight'].astype('float').apply(lambda x: round(x,2))
        return products_df


In [6]:
import re
df = convert_product_weights(df)
df.columns

/tmp/ipykernel_13963/1440517881.py:40: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  products_df.loc[0:,'weight'] = products_df.loc[0:,'weight'].astype('float').apply(lambda x: round(x,2))


Index(['Unnamed: 0', 'product_name', 'product_price', 'weight', 'category',
       'EAN', 'date_added', 'uuid', 'removed', 'product_code'],
      dtype='object')

In [20]:
def clean_products_data(products_df):
    # drop one of the indexing columns and reindex 
    products_df = products_df.drop(columns='Unnamed: 0').reindex()
    # Drop NULL values
    products_df = products_df[products_df.weight != 'nan']
    products_df = products_df.dropna()
    # Drop duplicates
    products_df = products_df.drop_duplicates()
    return products_df

In [21]:
df = convert_product_weights(df)
df = clean_products_data(df)
df.to_string('*.txt')

/tmp/ipykernel_208477/3544818218.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  products_df.loc[0:,'weight'] = products_df.loc[0:,'weight'].astype('float').apply(lambda x: round(x,2))
/tmp/ipykernel_208477/3544818218.py:38: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  products_df.loc[0:,'weight'] = products_df.loc[0:,'weight'].astype('float').apply(lambda x: round(x,2))


In [12]:
test = 'hey.csv'
test[-3:]

'csv'

In [5]:
import pandas as pd

In [11]:
card_table = pd.read_csv('unclean_credit_card.csv')
card_table

,Unnamed: 0,card_number,expiry_date,card_provider,date_payment_confirmed
0,0,30060773296197,09/26,Diners Club / Carte Blanche,2015-11-25
1,1,349624180933183,10/23,American Express,2001-06-18
2,2,3529023891650490,06/23,JCB 16 digit,2000-12-26
3,3,213142929492281,09/27,JCB 15 digit,2011-02-12
4,4,502067329974,10/25,Maestro,1997-03-13
...,...,...,...,...,...
15304,14,180036921556789,12/28,JCB 15 digit,1997-06-06
15305,15,180018030448512,11/24,JCB 15 digit,2004-06-16
15306,16,3569953313547220,04/24,JCB 16 digit,2020-02-05
15307,17,4444521712606810,06/27,VISA 16 digit,2008-06-16


In [15]:
card_table.query("card_number == '4971858637664481'")

,Unnamed: 0,card_number,expiry_date,card_provider,date_payment_confirmed


In [34]:
card_table.to_string('unclean_credit_card.txt')

In [35]:
import re
def clean_card_data(credit_card_df):
        # Reset indexing
        index = [row for row in range(0,len(credit_card_df))] 
        credit_card_df['index'] = index                         # new column
        credit_card_df = credit_card_df.set_index(['index'])    # replace index
        # Get rid of duplicates
        credit_card_df = credit_card_df.drop_duplicates()
        # Drop NULL values
        credit_card_df = credit_card_df[credit_card_df.card_number != 'NULL']
        credit_card_df = credit_card_df.dropna()
        # Infer datatypes
        credit_card_df['card_number'] = credit_card_df['card_number'].astype('str')     
        credit_card_df['card_provider'] = credit_card_df['card_provider'].astype('str')
        # Drop wrong values
        credit_card_df.loc[:,'card_number'] = credit_card_df.loc[:,'card_number'].astype('str').apply(lambda x:x.replace('?',''))
        credit_card_df = credit_card_df[credit_card_df['card_number'].str.isnumeric()]
        # Fix date formatting
        credit_card_df.loc[:,'expiry_date'] = \
        credit_card_df.loc[:,'expiry_date'].apply(pd.to_datetime, format='%m/%y')
        credit_card_df.loc[:,'date_payment_confirmed'] = \
        credit_card_df.loc[:,'date_payment_confirmed'].apply(pd.to_datetime,
                                                                infer_datetime_format=True,
                                                                errors='coerce')
        return credit_card_df

In [36]:
clean_card_table = clean_card_data(card_table)
clean_card_table.query("card_number == '4971858637664481'")

/tmp/ipykernel_73506/2228684045.py:19: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  credit_card_df.loc[:,'expiry_date'] = \
/tmp/ipykernel_73506/2228684045.py:21: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  credit_card_df.loc[:,'date_payment_confirmed'] = \


,Unnamed: 0,card_number,expiry_date,card_provider,date_payment_confirmed
index,,,,,
149,39,4971858637664481,2024-04-01,VISA 16 digit,2016-11-17


In [60]:
stores_table = pd.read_csv('unclean_stores_data.csv')
stores_table

,Unnamed: 0,index,address,longitude,lat,locality,store_code,staff_numbers,opening_date,store_type,latitude,country_code,continent
0,0,0,NaN,NaN,NaN,NaN,WEB-1388012W,325,2010-06-12,Web Portal,NaN,GB,Europe
1,1,1,"Flat 72W\nSally isle\nEast Deantown\nE7B 8EB, ...",51.62907,NaN,High Wycombe,HI-9B97EE4E,34,1996-10-25,Local,-0.74934,GB,Europe
2,2,2,"Heckerstraße 4/5\n50491 Säckingen, Landshut",48.52961,NaN,Landshut,LA-0772C7B9,92,2013-04-12,Super Store,12.16179,DE,Europe
3,3,3,"5 Harrison tunnel\nSouth Lydia\nWC9 2BE, Westbury",51.26,NaN,Westbury,WE-1DE82CEE,69,2014-01-02,Super Store,-2.1875,GB,Europe
4,4,4,Studio 6\nStephen landing\nSouth Simon\nB77 2W...,53.0233,NaN,Belper,BE-18074576,35,2019-09-09,Local,-1.48119,GB,Europe
...,...,...,...,...,...,...,...,...,...,...,...,...,...
445,445,445,"Flat 7\nStephanie lake\nMorrisside\nHP8 8LH, C...",50.76306,NaN,Cowes,CO-473A9FBB,94,2008-06-08,Super Store,-1.29772,GB,Europe
446,446,446,"Täschestraße 25\n39039 Nördlingen, Kirchlengern",52.2,NaN,Kirchlengern,KI-78096E8C,61,2005-05-12,Super Store,8.63333,DE,Europe
447,447,447,K0ODETRLS3,K8CXLZDP07,UXMWDMX1LC,3VHFDNP8ET,9D4LK7X4LZ,D23PCWSM6S,36IIMAQD58,NN04B3F6UQ,JZP8MIJTPZ,B3EH2ZGQAV,1WZB1TE1HL
448,448,448,"Studio 8\nMoss mall\nWest Linda\nM0E 6XR, High...",51.62907,NaN,High Wycombe,HI-EEA7AE62,33,1998-05-14,Local,-0.74934,GB,Europe


In [3]:
data_extractor = DataExtractor()
stores_df = data_extractor.retrieve_stores_data()
stores_df.to_string('unclean_stores_data.txt')
stores_df.to_csv('unclean_stores_data.csv')

In [5]:
stores_df.head()

,Unnamed: 0,index,address,longitude,lat,locality,store_code,staff_numbers,opening_date,store_type,latitude,country_code,continent
0,0,0,NaN,NaN,NaN,NaN,WEB-1388012W,325,2010-06-12,Web Portal,NaN,GB,Europe
1,1,1,"Flat 72W\nSally isle\nEast Deantown\nE7B 8EB, ...",51.62907,NaN,High Wycombe,HI-9B97EE4E,34,1996-10-25,Local,-0.74934,GB,Europe
2,2,2,"Heckerstraße 4/5\n50491 Säckingen, Landshut",48.52961,NaN,Landshut,LA-0772C7B9,92,2013-04-12,Super Store,12.16179,DE,Europe
3,3,3,"5 Harrison tunnel\nSouth Lydia\nWC9 2BE, Westbury",51.26,NaN,Westbury,WE-1DE82CEE,69,2014-01-02,Super Store,-2.1875,GB,Europe
4,4,4,Studio 6\nStephen landing\nSouth Simon\nB77 2W...,53.0233,NaN,Belper,BE-18074576,35,2019-09-09,Local,-1.48119,GB,Europe


In [7]:
stores_df[['store_type']].groupby('store_type').value_counts()

store_type
0RSNUU3DF5       1
50IB01SFAZ       1
B4KVQB3P5Y       1
Local          254
Mall Kiosk      51
NN04B3F6UQ       1
O0QJIRC943       1
Outlet          45
QP74AHEQT0       1
Super Store     89
Web Portal       1
X0FE7E2EOG       1
dtype: int64

In [8]:
stores_df.query("store_code=='BA-B4AED588'")

,Unnamed: 0,index,address,longitude,lat,locality,store_code,staff_numbers,opening_date,store_type,latitude,country_code,continent


In [51]:
def clean_store_data(stores_df):
        # Delete duplicate indexer column 
        stores_df = stores_df.iloc[:,1:]
        # Get rid of duplicates
        stores_df = stores_df.drop_duplicates()
        # Drop NULL values
        stores_df = stores_df[stores_df.address != 'N/A']
         # Get rid of rows with wrong values
        stores_df = stores_df.loc[stores_df['country_code'].isin(['GB','US','DE'])]
        # Remove escape characters
        stores_df.loc[:,'address'] =  stores_df.loc[:,'address'].astype('str').apply(lambda x:x.replace('\n', ','))
        # Fix values mixing letters with numbers
        stores_df.loc[:,'staff_numbers'] = stores_df['staff_numbers'].astype('str').apply(lambda x : re.sub('\D','',x))
        # Fix date formatting
        stores_df[['opening_date']] = \
        stores_df[['opening_date']].apply(pd.to_datetime,
                                            infer_datetime_format=True,
                                            errors='coerce')
        # Fix continent misspellings
        stores_df[['continent']] = stores_df[['continent']] \
                                    .apply(lambda x:x.replace('eeEurope','Europe')) \
                                    .apply(lambda x:x.replace('eeAmerica','America'))
        return  stores_df

In [69]:
user_df = pd.read_csv('unclean_user_data.csv')
user_df

,Unnamed: 0,index,first_name,last_name,date_of_birth,company,email_address,address,country,country_code,phone_number,join_date,user_uuid
0,0,0,Sigfried,Noack,1990-09-30,Heydrich Junitz KG,rudi79@winkler.de,Zimmerstr. 1/0\n59015 Gießen,Germany,DE,+49(0) 047905356,2018-10-10,93caf182-e4e9-4c6e-bebb-60a1a9dcf9b8
1,1,1,Guy,Allen,1940-12-01,Fox Ltd,rhodesclifford@henderson.com,Studio 22a\nLynne terrace\nMcCarthymouth\nTF0 9GH,United Kingdom,GB,(0161) 496 0674,2001-12-20,8fe96c3a-d62d-4eb5-b313-cf12d9126a49
2,2,2,Harry,Lawrence,1995-08-02,"Johnson, Jones and Harris",glen98@bryant-marshall.co.uk,92 Ann drive\nJoanborough\nSK0 6LR,United Kingdom,GB,+44(0)121 4960340,2016-12-16,fc461df4-b919-48b2-909e-55c95a03fe6b
3,3,3,Darren,Hussain,1972-09-23,Wheeler LLC,daniellebryan@thompson.org,19 Robinson meadow\nNew Tracy\nW22 2QG,United Kingdom,GB,(0306) 999 0871,2004-02-23,6104719f-ef14-4b09-bf04-fb0c4620acb0
4,4,4,Garry,Stone,1952-12-20,Warner Inc,billy14@long-warren.com,3 White pass\nHunterborough\nNN96 4UE,United Kingdom,GB,0121 496 0225,2006-09-01,9523a6d3-b2dd-4670-a51a-36aebc89f579
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15315,15315,14913,Stephen,Jenkins,1943-08-09,"Thornton, Carroll and Newman",s.jenkins@smith.com,Studio 41I\nJones lodge\nOliviaborough\nE8 3DU,United Kingdom,GB,+44(0)292018946,2016-04-15,2bd3a12f-a92d-4cdd-b99c-fc70572db302
15316,15316,14994,Stephen,Smith,1948-08-20,Robinson-Harris,s.smith@smith.com,530 Young parkway\nMillsfurt\nL4G 7NX,United Kingdom,GB,+44(0)1144960977,2020-07-20,d234c04b-c07c-46a5-a902-526f91478ecc
15317,15317,15012,Stephen,Losekann,1940-10-09,Rosenow,s.losekann@smith.com,Viviane-Fritsch-Straße 3/5\n15064 Bad Liebenwerda,Germany,DE,02984 08192,2021-03-07,1a0a8b7b-7c17-42d8-a946-8a85d5495651
15318,15318,15269,Stephen,Rivera,1952-06-04,"Taylor, Fry and Jones",s.rivera@smith.com,"660 Ross Falls Suite 357\nAnthonymouth, MA 09610",United States,US,239.711.3836,2011-01-03,187fe06e-bd5f-4381-af2f-d7ac37ca7572


In [75]:
len(user_df)

15320

In [70]:
user_df.query("user_uuid =='bd3e3bc5-3a25-40dd-ac0f-3f2aaf5814df'")

,Unnamed: 0,index,first_name,last_name,date_of_birth,company,email_address,address,country,country_code,phone_number,join_date,user_uuid
2597,2597,2597,Peter,Mitchell,1942-01-11,"Jennings, Taylor and Davies",mlambert@riley.com,849 Collins camp\nSouth Francisland\nAL80 8TD,United Kingdom,GGB,0121 4960043,1995-07-15,bd3e3bc5-3a25-40dd-ac0f-3f2aaf5814df


In [93]:
user_df.loc[~user_df['country_code'].isin(['GB','US','DE'])]

,Unnamed: 0,index,first_name,last_name,date_of_birth,company,email_address,address,country,country_code,phone_number,join_date,user_uuid
752,752,752,PYCLKLLC7I,W350SCUD6R,KBTI7FI7Y3,R7IZUNSQX0,3Q791B3VIY,YW2YXLOQ5J,I7G4DMDZOZ,VSM4IZ4EL3,A4Q4HQBI3I,JJ2PDVNPRO,W43MSCMQ88
866,866,867,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1022,1022,1023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1046,1046,1047,GI4C78KWH0,UTB5PPYFG8,OFH8YGZJWN,CA1XGS8GZW,7HSZB429UK,63GXGYR3XL,AJ1ENKS3QL,QVUW9JSKY3,64ZO0ONUQO,AHN6EKASH3,BUE34OU973
1805,1805,1807,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2103,2103,2103,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2437,2437,2439,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2597,2597,2597,Peter,Mitchell,1942-01-11,"Jennings, Taylor and Davies",mlambert@riley.com,849 Collins camp\nSouth Francisland\nAL80 8TD,United Kingdom,GGB,0121 4960043,1995-07-15,bd3e3bc5-3a25-40dd-ac0f-3f2aaf5814df
2739,2739,6526,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2764,2764,2764,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [86]:
def clean_user_data(users_df):
        # Get rid of duplicates
        users_df = users_df.drop_duplicates()
        # Get rid of NULL values
        users_df = users_df[users_df.first_name != 'NULL']    
        users_df = users_df.dropna()
        # Get rid of rows with wrong values
        users_df.loc[:,'country_code'] = users_df.loc[:,'country_code'].astype('string').apply(lambda x : x.replace('GGB','GB'))
        users_df = users_df.loc[users_df['country_code'].isin(['GB','US','DE'])]   
        # Fix date formatting
        date_cols = ['date_of_birth','join_date']
        for date_col in date_cols:
            users_df.loc[:,date_col] = users_df.loc[:,date_col].apply(pd.to_datetime, 
                                            infer_datetime_format=True, 
                                            errors='coerce')
        # Remove escape characters
        users_df.loc[:,'address'] = users_df.loc[:,'address'].apply(lambda x:x.replace('\n', ','))
        return users_df

In [87]:
clean_user_df = clean_user_data(user_df)

/tmp/ipykernel_73506/1284253201.py:13: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  users_df.loc[:,date_col] = users_df.loc[:,date_col].apply(pd.to_datetime,


In [76]:
len(clean_user_df)

15278

In [88]:
clean_user_df.query("user_uuid =='bd3e3bc5-3a25-40dd-ac0f-3f2aaf5814df'")

,Unnamed: 0,index,first_name,last_name,date_of_birth,company,email_address,address,country,country_code,phone_number,join_date,user_uuid
2597,2597,2597,Peter,Mitchell,1942-01-11,"Jennings, Taylor and Davies",mlambert@riley.com,"849 Collins camp,South Francisland,AL80 8TD",United Kingdom,GB,0121 4960043,1995-07-15,bd3e3bc5-3a25-40dd-ac0f-3f2aaf5814df


In [89]:
clean_user_df.query("email_address =='mlambert@riley.com'")

,Unnamed: 0,index,first_name,last_name,date_of_birth,company,email_address,address,country,country_code,phone_number,join_date,user_uuid
2597,2597,2597,Peter,Mitchell,1942-01-11,"Jennings, Taylor and Davies",mlambert@riley.com,"849 Collins camp,South Francisland,AL80 8TD",United Kingdom,GB,0121 4960043,1995-07-15,bd3e3bc5-3a25-40dd-ac0f-3f2aaf5814df
